In [26]:
import geopandas as gpd
df = gpd.read_file('/data/acker/ALA/paper2/all_variables.shp')
df = df.rename(columns={'monitor__1':'monitor_coverage_pct', 'classifica':'classification', 'Design Val':'Design Value', 'monitor_co':'monitor_count',
                        'fire_regio':'fire_region', 'mountain_r':'mountain_region', 'desert_reg':'desert_region', 'urban_cate':'urban_category'})
#upload shapefile of U.S. counties
counties = gpd.read_file('/data/acker/shapefiles/cb_2020_us_county_500k.shp')
# List of state abbreviations for CONUS, Alaska (AK), and Hawaii (HI)
states_to_include = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'DC',
    'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
    'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
    'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
    'WI', 'WY'
]

# Filter counties to only include rows where STUSPS is in the specified list
counties_conus = counties[counties['STUSPS'].isin(states_to_include)]

# Display the filtered counties
counties_conus = counties_conus.drop(["COUNTYNS", 'NAMELSAD', 'LSAD', 'ALAND', 'AWATER', 'AFFGEOID'], axis=1)
counties_conus = counties_conus.to_crs(df.crs)
df = df.drop(columns='geometry')


# Merge with both GEOID and geometry
new = df.merge(counties_conus[['GEOID', 'geometry']], on='GEOID', how='inner')

# Recast as GeoDataFrame
ranked_correct = gpd.GeoDataFrame(new, geometry='geometry', crs=counties_conus.crs)
df = ranked_correct

In [27]:
df.columns

Index(['GEOID', 'PM25_90th', 'Design Value', 'classification', 'diff',
       'abs_diff', 'monitor_count', 'cdv_bin', 'monitor_coverage_pct',
       'dist_km', 'size', 'fire_region', 'mountain_region', 'desert_region',
       'urban_category', 'geometry'],
      dtype='object')

In [28]:
import numpy as np

df["monitor_bin"] = np.where(df["monitor_count"] <= 2, "<=2 monitors", ">2 monitors")
df["monitor_coverage_category"] = np.where(
    (df["monitor_coverage_pct"] < 0.05) | (df["monitor_coverage_pct"] > 0.1),
    "High risk (<0.05 or >0.1)",
    "Low risk (0.05–0.1)"
)
df['CDV_binned'] = np.where(
    (df["cdv_bin"] == '<7') | (df["cdv_bin"] == '>10'),
    "High risk (< 7 or > 10)",
    "Low risk (7–10)"
)

In [29]:
df

,GEOID,PM25_90th,Design Value,classification,diff,abs_diff,monitor_count,cdv_bin,monitor_coverage_pct,dist_km,size,fire_region,mountain_region,desert_region,urban_category,geometry,monitor_bin,monitor_coverage_category,CDV_binned
0,02090,14.333333,12.1,TP,2.233333,2.233333,3,>10,0.008092,85.118385,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-148.66326 64.59079, -148.64821 64.5...",>2 monitors,High risk (<0.05 or >0.1),High risk (< 7 or > 10)
1,06063,14.200000,14.0,TP,0.200000,0.200000,1,>10,0.013591,60.606568,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-121.49703 40.43702, -121.49487 40.4...",<=2 monitors,High risk (<0.05 or >0.1),High risk (< 7 or > 10)
2,06107,12.933334,15.7,TP,-2.766666,2.766666,1,>10,0.007784,100.833209,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-119.56647 36.49434, -119.56366 36.4...",<=2 monitors,High risk (<0.05 or >0.1),High risk (< 7 or > 10)
3,06023,12.633334,6.9,FP,5.733334,5.733334,1,<7,0.009894,28.714886,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-124.4086 40.4432, -124.39664 40.462...",<=2 monitors,High risk (<0.05 or >0.1),High risk (< 7 or > 10)
4,06029,12.599999,16.2,TP,-3.600001,3.600001,5,>10,0.023405,82.031806,large,Fire,Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-120.19437 35.78936, -120.00308 35.7...",>2 monitors,High risk (<0.05 or >0.1),High risk (< 7 or > 10)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,02020,4.133333,5.1,TN,-0.966667,0.966667,1,<7,0.020235,66.108117,small,Non-Fire,Mountain,Non-Desert,Non-Urban (<50%),"MULTIPOLYGON (((-150.07348 61.15834, -150.0691...",<=2 monitors,High risk (<0.05 or >0.1),High risk (< 7 or > 10)
532,02110,3.900000,4.6,TN,-0.700000,0.700000,1,<7,0.018594,73.544905,large,Non-Fire,Mountain,Non-Desert,Non-Urban (<50%),"MULTIPOLYGON (((-134.66932 58.33327, -134.6675...",<=2 monitors,High risk (<0.05 or >0.1),High risk (< 7 or > 10)
533,15001,3.633333,4.4,TN,-0.766667,0.766667,5,<7,0.054825,81.864071,large,Fire,Non-Mountain,Non-Desert,Non-Urban (<50%),"POLYGON ((-156.06147 19.72813, -156.06076 19.7...",>2 monitors,Low risk (0.05–0.1),High risk (< 7 or > 10)
534,15009,3.556667,4.0,TN,-0.443333,0.443333,1,<7,0.035920,78.820617,small,Fire,Non-Mountain,Non-Desert,Non-Urban (<50%),"MULTIPOLYGON (((-156.69742 20.91637, -156.6957...",<=2 monitors,High risk (<0.05 or >0.1),High risk (< 7 or > 10)


In [30]:
df.columns

Index(['GEOID', 'PM25_90th', 'Design Value', 'classification', 'diff',
       'abs_diff', 'monitor_count', 'cdv_bin', 'monitor_coverage_pct',
       'dist_km', 'size', 'fire_region', 'mountain_region', 'desert_region',
       'urban_category', 'geometry', 'monitor_bin',
       'monitor_coverage_category', 'CDV_binned'],
      dtype='object')

In [ ]:

#df = df[df['classification'] == 'FN']
#df = df[df['classification'] == 'FP'] # | (df['classification'] == 'FN')]
df = df[(df['classification'] == 'FP') | (df['classification'] == 'FN')]

In [32]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu

# ---- Configure your high/low labels for each variable ----
risk_labels = {
    "monitor_bin": {
        "high": "<=2 monitors",
        "low":  ">2 monitors",
    },
    "CDV_binned": {
        "high": "High risk (< 7 or > 10)",
        "low":  "Low risk (7–10)",
    },
    "size": {
        "high": "large",
        "low":  "small",
    },
    "fire_region": {
        "high": "Fire",
        "low":  "Non-Fire",
    },
    "mountain_region": {
        "high": "Mountain",
        "low":  "Non-Mountain",
    },
    "desert_region": {
        "high": "Desert",
        "low":  "Non-Desert",
    },
    "urban_category": {
        "high": "Non-Urban (<50%)",
        "low":  "Urban (≥50%)",
    },
    "monitor_coverage_category": {
        "high": "High risk (<0.05 or >0.1)",
        "low":  "Low risk (0.05–0.1)",
    },
}

rows = []

for var, labels in risk_labels.items():
    high_lab = labels["high"]
    low_lab  = labels["low"]

    # Keep only rows that are explicitly high or low for this variable
    sub = df[df[var].isin([high_lab, low_lab])].copy()

    # Pull abs_diff for each group
    high = sub.loc[sub[var] == high_lab, "abs_diff"].dropna()
    low  = sub.loc[sub[var] == low_lab,  "abs_diff"].dropna()

    n_high = high.size
    n_low  = low.size

    # Default outputs
    pval = np.nan
    U = np.nan

    # Mann–Whitney requires at least 1 observation in each group
    if n_high > 0 and n_low > 0:
        U, pval = mannwhitneyu(high, low, alternative="two-sided", method="auto")

    # Summaries (report medians for non-parametric test; also include means for reference)
    high_median = np.nan if n_high == 0 else high.median()
    low_median  = np.nan if n_low  == 0 else low.median()
    diff_median = (high_median - low_median) if (n_high > 0 and n_low > 0) else np.nan

    high_mean = np.nan if n_high == 0 else high.mean()
    low_mean  = np.nan if n_low  == 0 else low.mean()
    diff_mean = (high_mean - low_mean) if (n_high > 0 and n_low > 0) else np.nan

    rows.append({
        "Variable": var,
        "High label": high_lab,
        "Low label": low_lab,
        "n_high": n_high,
        "n_low": n_low,
        # "Original 'abs_diff'": summarize with medians (non-parametric) + means
        "high_abs_diff_median": high_median,
        "low_abs_diff_median": low_median,
        "Δ_median_high_minus_low": diff_median,
        "high_abs_diff_mean": high_mean,
        "low_abs_diff_mean": low_mean,
        "Δ_mean_high_minus_low": diff_mean,
        # Test stats
        "U_stat": U,
        "p_value": pval
    })

out = pd.DataFrame(rows).sort_values("p_value", na_position="last").reset_index(drop=True)

# If you only want the essentials, you could subset like this:
# out = out[[
#     "Variable", "High label", "Low label", "n_high", "n_low",
#     "high_abs_diff_median", "low_abs_diff_median", "Δ_median_high_minus_low", "p_value"
# ]]

out


,Variable,High label,Low label,n_high,n_low,high_abs_diff_median,low_abs_diff_median,Δ_median_high_minus_low,high_abs_diff_mean,low_abs_diff_mean,Δ_mean_high_minus_low,U_stat,p_value
0,CDV_binned,High risk (< 7 or > 10),Low risk (7–10),19,33,2.233334,0.866667,1.366667,2.636842,1.134344,1.502499,557.0,0.000004
1,size,large,small,14,38,2.500000,1.033333,1.466666,2.754762,1.288597,1.466165,451.0,0.000141
2,fire_region,Fire,Non-Fire,20,32,2.200000,0.966667,1.233333,2.375000,1.251042,1.123958,490.0,0.001432
3,urban_category,Non-Urban (<50%),Urban (≥50%),46,6,1.700000,0.583333,1.116666,1.827174,0.580556,1.246618,232.0,0.005048
4,desert_region,Desert,Non-Desert,3,49,4.300000,1.200000,3.100000,3.755556,1.556463,2.199093,126.0,0.036742
5,mountain_region,Mountain,Non-Mountain,22,30,1.983333,1.116667,0.866667,1.927273,1.504445,0.422828,417.0,0.109128
6,monitor_bin,<=2 monitors,>2 monitors,46,6,1.550000,0.858334,0.691667,1.702536,1.536111,0.166425,178.0,0.265310
7,monitor_coverage_category,High risk (<0.05 or >0.1),Low risk (0.05–0.1),35,17,1.766666,1.200000,0.566666,1.837143,1.366667,0.470476,340.0,0.412615


In [33]:
#out.to_csv('/data/acker/ALA/paper2/NPNNp-values.csv')
#out.to_csv('/data/acker/ALA/paper2/NPp-values.csv')
out.to_csv('/data/acker/ALA/paper2/NNp-values.csv')

In [ ]:
import pandas as pd

# Variables to summarize
group_vars = [
    "monitor_bin",
    "CDV_binned",
    "size",
    "fire_region",
    "mountain_region",
    "desert_region",
    "urban_category",
    "monitor_coverage_category",
]

# Subsets
df_non_aligned = df[df["classification"].isin(["FP", "FN"])]
df_fp = df[df["classification"] == "FP"]
df_fn = df[df["classification"] == "FN"]

def summarize_abs_diff(df_subset: pd.DataFrame, subset_name: str) -> pd.DataFrame:
    rows = []
    for var in group_vars:
        temp = df_subset[[var, "abs_diff"]].dropna()
        if temp.empty:
            continue

        g = (
            temp.groupby(var, dropna=False)["abs_diff"]
                .agg(mean="mean", median="median", count="size")
                .reset_index()
        )
        # Standardize the category column name
        g = g.rename(columns={var: "Category"})
        # Attach identifiers and tidy types
        g.insert(0, "Subset", subset_name)
        g.insert(1, "Variable", var)
        g["mean"] = g["mean"].round(3)
        g["median"] = g["median"].round(3)
        g["count"] = g["count"].astype(int)
        rows.append(g)

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(
        columns=["Subset","Variable","Category","mean","median","count"]
    )

# Build tables
summary_non_aligned = summarize_abs_diff(df_non_aligned, "FP+FN")
summary_fp = summarize_abs_diff(df_fp, "FP")
summary_fn = summarize_abs_diff(df_fn, "FN")

# Combine
summary_all = pd.concat([summary_non_aligned, summary_fp, summary_fn], ignore_index=True)

# Reorder columns (now Category exists uniformly)
summary_all = summary_all[["Subset", "Variable", "Category", "mean", "median", "count"]]

# summary_all.to_csv("abs_diff_summary_with_monitor_coverage.csv", index=False)
summary_all

,Subset,Variable,Category,mean,median,count
0,FP+FN,monitor_bin,<=2 monitors,1.553,1.333,89
1,FP+FN,monitor_bin,>2 monitors,1.244,0.492,8
2,FP+FN,cdv_bin,7–10,1.164,0.933,75
3,FP+FN,cdv_bin,<7,3.589,2.633,3
4,FP+FN,cdv_bin,>10,2.637,2.233,19
5,FP+FN,size,large,2.833,2.633,17
6,FP+FN,size,small,1.250,1.033,80
7,FP+FN,fire_region,Fire,2.334,2.183,30
8,FP+FN,fire_region,Non-Fire,1.166,1.000,67
9,FP+FN,mountain_region,Mountain,1.712,1.333,39


In [45]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal

# -----------------------------
# Variables to summarize
# -----------------------------
group_vars = [
    "monitor_bin",
    "cdv_bin",
    "size",
    "fire_region",
    "mountain_region",
    "desert_region",
    "urban_category",
    "monitor_coverage_category",
]

# -----------------------------
# Subsets
# -----------------------------
df_non_aligned = df[df["classification"].isin(["FP", "FN"])]
df_fp = df[df["classification"] == "FP"]
df_fn = df[df["classification"] == "FN"]

# -----------------------------
# Helpers
# -----------------------------
def kruskal_eta2(temp: pd.DataFrame, group_col: str, value_col: str):
    temp = temp[[group_col, value_col]].dropna()
    if temp.empty:
        return np.nan, np.nan, np.nan
    groups = temp[group_col].unique()
    if len(groups) < 2:
        return np.nan, np.nan, np.nan
    data_groups = [temp.loc[temp[group_col] == g, value_col].values for g in groups]
    H, p = kruskal(*data_groups)
    n = len(temp)
    k = len(groups)
    eta2 = (H - k + 1) / (n - k) if n > k else np.nan
    return round(H, 3), round(p, 6), round(eta2, 4)

def summarize_abs_diff(df_subset: pd.DataFrame, subset_name: str) -> pd.DataFrame:
    rows = []
    for var in group_vars:
        # keep both the grouping column and abs_diff; drop rows missing either
        temp = df_subset[[var, "abs_diff"]].dropna()
        if temp.empty:
            continue

        # descriptive stats by category
        g = (
            temp.groupby(var, dropna=False)["abs_diff"]
                .agg(mean="mean", median="median", count="size")
                .reset_index()
        )
        # rename the grouping column to a common name
        g = g.rename(columns={var: "Category"})
        # add test results (one p and η² per variable; repeat for each category row)
        H, p, eta2 = kruskal_eta2(temp, var, "abs_diff")
        g.insert(0, "Subset", subset_name)
        g.insert(1, "Variable", var)
        g["p-value"] = p
        g["η²"] = eta2
        # tidy types/rounding
        g["mean"] = g["mean"].round(3)
        g["median"] = g["median"].round(3)
        g["count"] = g["count"].astype(int)
        rows.append(g)

    if rows:
        out = pd.concat(rows, ignore_index=True)
        # ensure expected columns exist
        return out[["Subset", "Variable", "Category", "mean", "median", "count", "p-value", "η²"]]
    else:
        return pd.DataFrame(columns=["Subset","Variable","Category","mean","median","count","p-value","η²"])

# -----------------------------
# Build tables
# -----------------------------
summary_non_aligned = summarize_abs_diff(df_non_aligned, "FP+FN")
summary_fp          = summarize_abs_diff(df_fp,         "FP")
summary_fn          = summarize_abs_diff(df_fn,         "FN")

summary_all = pd.concat([summary_non_aligned, summary_fp, summary_fn], ignore_index=True)

summary_all

,Subset,Variable,Category,mean,median,count,p-value,η²
0,FP+FN,monitor_bin,<=2 monitors,1.553,1.333,89,0.078867,0.0220
1,FP+FN,monitor_bin,>2 monitors,1.244,0.492,8,0.078867,0.0220
2,FP+FN,cdv_bin,7–10,1.164,0.933,75,0.000000,0.3219
3,FP+FN,cdv_bin,<7,3.589,2.633,3,0.000000,0.3219
4,FP+FN,cdv_bin,>10,2.637,2.233,19,0.000000,0.3219
5,FP+FN,size,large,2.833,2.633,17,0.000006,0.2042
6,FP+FN,size,small,1.250,1.033,80,0.000006,0.2042
7,FP+FN,fire_region,Fire,2.334,2.183,30,0.000016,0.1856
8,FP+FN,fire_region,Non-Fire,1.166,1.000,67,0.000016,0.1856
9,FP+FN,mountain_region,Mountain,1.712,1.333,39,0.196616,0.0070


In [46]:
summary_all.to_csv('/data/acker/ALA/paper2/p-value-mean-diffs.csv')

In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu, kruskal

# Example: assuming your dataframe is called df
# Replace with your actual DataFrame variable
# df = pd.read_csv("your_file.csv")

# List of variables to test (exclude numeric columns like 'diff' or 'abs_diff')
group_vars = [
    "monitor_bin",
    "cdv_bin",
    "size",
    "fire_region",
    "mountain_region",
    "desert_region",
    "urban_category",
    "monitor_coverage_category"
]

results = []

for var in group_vars:
    # Drop missing values for this variable
    temp = df[[var, "abs_diff"]].dropna()
    groups = temp[var].unique()
    
    # Skip variables with only one group
    if len(groups) < 2:
        continue

    # Prepare data by group
    data_groups = [temp.loc[temp[var] == g, "abs_diff"] for g in groups]
    
    # Choose test based on number of unique groups
    if len(groups) == 2:
        test_name = "Mann–Whitney U"
        stat, p = mannwhitneyu(data_groups[0], data_groups[1], alternative="two-sided")
    else:
        test_name = "Kruskal–Wallis"
        stat, p = kruskal(*data_groups)
    
    # Store results
    results.append({
        "Variable": var,
        "Groups": len(groups),
        "Test": test_name,
        "Statistic": round(stat, 3),
        "p-value": round(p, 4)
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df.sort_values("p-value", inplace=True)

results_df

,Variable,Groups,Test,Statistic,p-value
1,cdv_bin,3,Kruskal–Wallis,61.586,0.0000
2,size,2,Mann–Whitney U,26895.000,0.0000
3,fire_region,2,Mann–Whitney U,32829.500,0.0009
5,desert_region,2,Mann–Whitney U,2073.000,0.0057
4,mountain_region,2,Mann–Whitney U,37518.000,0.2414
0,monitor_bin,2,Mann–Whitney U,15369.500,0.2476
6,urban_category,2,Mann–Whitney U,16571.000,0.9137


In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu, kruskal
import numpy as np

# ---- Filter for NP and NN counties only ----
df_subset = df[df["classification"].isin(["FP"])].copy()

# ---- Define the variables to test ----
group_vars = [
    "monitor_bin",
    "cdv_bin",
    "size",
    "fire_region",
    "mountain_region",
    "desert_region",
    "urban_category",
    "monitor_coverage_category"
]

results = []

for var in group_vars:
    # Drop missing values for this variable
    temp = df_subset[[var, "abs_diff"]].dropna()
    groups = temp[var].unique()
    
    # Skip variables with only one group
    if len(groups) < 2:
        continue

    # Prepare data by group
    data_groups = [temp.loc[temp[var] == g, "abs_diff"] for g in groups]
    
    # Choose test based on number of unique groups
    if len(groups) == 2:
        test_name = "Mann–Whitney U"
        stat, p = mannwhitneyu(data_groups[0], data_groups[1], alternative="two-sided")
    else:
        test_name = "Kruskal–Wallis"
        stat, p = kruskal(*data_groups)
    
    # Store results
    results.append({
        "Variable": var,
        "Groups": len(groups),
        "Test": test_name,
        "Statistic": round(stat, 3),
        "p-value": round(p, 4)
    })

# ---- Convert to DataFrame and sort by p-value ----
results_df = pd.DataFrame(results)
results_df.sort_values("p-value", inplace=True)

results_df


,Variable,Groups,Test,Statistic,p-value
1,cdv_bin,2,Mann–Whitney U,121.0,0.0023
3,fire_region,2,Mann–Whitney U,273.0,0.0063
0,monitor_bin,2,Mann–Whitney U,75.0,0.0848
2,size,2,Mann–Whitney U,100.0,0.1001
5,desert_region,2,Mann–Whitney U,39.0,0.2667
6,urban_category,2,Mann–Whitney U,247.0,0.4741
4,mountain_region,2,Mann–Whitney U,233.5,0.9354


In [35]:
import pandas as pd
from scipy.stats import kruskal

# Filter for NP + NN counties
df_subset = df[df["classification"].isin(["FP", "FN"])].copy()

group_vars = [
    "monitor_bin",
    "cdv_bin",
    "size",
    "fire_region",
    "mountain_region",
    "desert_region",
    "urban_category",
    "monitor_coverage_category"
]

results = []

for var in group_vars:
    temp = df_subset[[var, "abs_diff"]].dropna()
    groups = temp[var].unique()
    
    if len(groups) < 2:
        continue

    # Create a list of abs_diff arrays by group
    data_groups = [temp.loc[temp[var] == g, "abs_diff"] for g in groups]

    # Perform Kruskal-Wallis test (works for 2 or more groups)
    stat, p = kruskal(*data_groups)

    results.append({
        "Variable": var,
        "Groups": len(groups),
        "Test": "Kruskal–Wallis",
        "Statistic": round(stat, 3),
        "p-value": round(p, 4)
    })

results_df = pd.DataFrame(results).sort_values("p-value")
results_df

,Variable,Groups,Test,Statistic,p-value
1,cdv_bin,3,Kruskal–Wallis,32.254,0.0000
2,size,2,Kruskal–Wallis,20.400,0.0000
3,fire_region,2,Kruskal–Wallis,18.630,0.0000
6,urban_category,2,Kruskal–Wallis,6.413,0.0113
7,monitor_coverage_category,2,Kruskal–Wallis,3.204,0.0735
0,monitor_bin,2,Kruskal–Wallis,3.088,0.0789
4,mountain_region,2,Kruskal–Wallis,1.667,0.1966
5,desert_region,2,Kruskal–Wallis,1.306,0.2530


In [36]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal

# Optional: pip install scikit-posthocs
import scikit_posthocs as sp

# ---- restrict to NP + NN counties (as you’ve been doing) ----
df_sub = df[df["classification"].isin(["TP", "TN"])].copy()

# ---- choose your grouping variables here ----
group_vars = [
    "monitor_bin",
    "cdv_bin",
    "size",
    "fire_region",
    "mountain_region",
    "desert_region",
    "urban_category",
    "monitor_coverage_category"
    # add/remove as needed
]

# Container for overall Kruskal results
kruskal_rows = []

# Also collect medians/IQRs for each variable/group for context
desc_rows = []

for var in group_vars:
    # Keep rows with both var and diff present
    temp = df_sub[[var, "diff"]].dropna()
    if temp.empty:
        continue

    # Ensure categorical dtype (keeps stable group ordering in outputs)
    if not pd.api.types.is_categorical_dtype(temp[var]):
        temp[var] = temp[var].astype("category")

    groups = temp[var].cat.categories.tolist() if pd.api.types.is_categorical_dtype(temp[var]) else sorted(temp[var].unique())

    # Skip variables with <2 groups present
    if len(groups) < 2:
        continue

    # Build list of series per group
    data_groups = [temp.loc[temp[var] == g, "diff"] for g in groups]

    # --- Kruskal–Wallis (works for 2+ groups) ---
    H, p_kw = kruskal(*data_groups)
    kruskal_rows.append({
        "Variable": var,
        "Groups": len(groups),
        "Test": "Kruskal–Wallis",
        "Statistic (H)": round(float(H), 3),
        "p-value": round(float(p_kw), 6),
        "Significant (p<0.05)": p_kw < 0.05
    })

    # --- Summaries: median and IQR per group ---
    for g in groups:
        vals = temp.loc[temp[var] == g, "diff"]
        q1, med, q3 = np.percentile(vals, [25, 50, 75])
        desc_rows.append({
            "Variable": var,
            "Group": g,
            "N": int(vals.shape[0]),
            "Median_diff": round(float(med), 3),
            "IQR_diff": f"{round(float(q1),3)}–{round(float(q3),3)}"
        })

    # --- If significant AND multi-group, run Dunn's post-hoc with Bonferroni ---
    if (p_kw < 0.05) and (len(groups) > 2):
        # Dunn’s expects long-form data
        dunn_df = sp.posthoc_dunn(temp, val_col="diff", group_col=var, p_adjust="bonferroni")
        # Save per-variable posthoc matrix
        dunn_df.index.name = "Group"
    

# Tidy outputs
kruskal_df = pd.DataFrame(kruskal_rows).sort_values("p-value", ascending=True)
desc_df = pd.DataFrame(desc_rows)


# Show the key table
kruskal_df


/tmp/ipykernel_489721/521170566.py:37: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(temp[var]):
/tmp/ipykernel_489721/521170566.py:40: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  groups = temp[var].cat.categories.tolist() if pd.api.types.is_categorical_dtype(temp[var]) else sorted(temp[var].unique())
/tmp/ipykernel_489721/521170566.py:37: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(temp[var]):
/tmp/ipykernel_489721/521170566.py:40: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  groups = temp[var].

,Variable,Groups,Test,Statistic (H),p-value,Significant (p<0.05)
0,monitor_bin,2,Kruskal–Wallis,26.002,0.000000,True
1,cdv_bin,3,Kruskal–Wallis,89.619,0.000000,True
2,size,2,Kruskal–Wallis,6.040,0.013989,True
3,fire_region,2,Kruskal–Wallis,3.261,0.070927,False
7,monitor_coverage_category,2,Kruskal–Wallis,2.101,0.147160,False
6,urban_category,2,Kruskal–Wallis,1.971,0.160329,False
5,desert_region,2,Kruskal–Wallis,0.071,0.789265,False
4,mountain_region,2,Kruskal–Wallis,0.030,0.862850,False


In [21]:
dunn_df

,7–10,<7,>10
Group,,,
7–10,1.000000e+00,0.013491,7.858527e-07
<7,1.349066e-02,1.000000,1.000000e+00
>10,7.858527e-07,1.000000,1.000000e+00


In [37]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal

# Filter to NP + NN counties
df_sub = df[df["classification"].isin(["FP", "FN"])].copy()

group_vars = [
    "monitor_bin",
    "cdv_bin",
    "size",
    "fire_region",
    "mountain_region",
    "desert_region",
    "urban_category",
    "monitor_coverage_category"
]

results = []

for var in group_vars:
    temp = df_sub[[var, "abs_diff"]].dropna()
    groups = temp[var].unique()
    if len(groups) < 2:
        continue

    # Create list of abs_diff arrays per group
    data_groups = [temp.loc[temp[var] == g, "abs_diff"] for g in groups]

    # Run Kruskal–Wallis
    H, p = kruskal(*data_groups)

    # --- Compute effect size (eta-squared) ---
    n = len(temp)              # total observations
    k = len(groups)            # number of groups
    eta_sq = (H - k + 1) / (n - k) if n > k else np.nan

    results.append({
        "Variable": var,
        "Groups": k,
        "Test": "Kruskal–Wallis",
        "Statistic (H)": round(float(H), 3),
        "p-value": round(float(p), 6),
        "η² (Effect Size)": round(float(eta_sq), 4),
        "Significant (p<0.05)": p < 0.05
    })

results_df = pd.DataFrame(results).sort_values("p-value")
results_df

,Variable,Groups,Test,Statistic (H),p-value,η² (Effect Size),Significant (p<0.05)
1,cdv_bin,3,Kruskal–Wallis,32.254,0.000000,0.3219,True
2,size,2,Kruskal–Wallis,20.400,0.000006,0.2042,True
3,fire_region,2,Kruskal–Wallis,18.630,0.000016,0.1856,True
6,urban_category,2,Kruskal–Wallis,6.413,0.011327,0.0570,True
7,monitor_coverage_category,2,Kruskal–Wallis,3.204,0.073475,0.0232,False
0,monitor_bin,2,Kruskal–Wallis,3.088,0.078867,0.0220,False
4,mountain_region,2,Kruskal–Wallis,1.667,0.196616,0.0070,False
5,desert_region,2,Kruskal–Wallis,1.306,0.253035,0.0032,False


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal

# ---- subset definitions ----
subsets = {
    "ALL": df.index,
    "TP_TN": df.index[df["classification"].isin(["TP","TN"])],
    "FP_FN": df.index[df["classification"].isin(["FP","FN"])],
    "TP": df.index[df["classification"] == "TP"],
    "TN": df.index[df["classification"] == "TN"],
    "FP": df.index[df["classification"] == "FP"],
    "FN": df.index[df["classification"] == "FN"],
}

group_vars = [
    "monitor_bin",
    "cdv_bin",
    "size",
    "fire_region",
    "mountain_region",
    "desert_region",
    "urban_category",
    "monitor_coverage_category"
]

def run_kw(df_sub, subset_name):
    out = []
    for var in group_vars:
        temp = df_sub[[var, "abs_diff"]].dropna()
        groups = temp[var].unique()
        if len(groups) < 2:
            continue

        # Kruskal–Wallis test
        data_groups = [temp.loc[temp[var]==g, "abs_diff"] for g in groups]
        H, p = kruskal(*data_groups)

        # η² effect size
        n = len(temp)
        k = len(groups)
        eta2 = (H - k + 1) / (n - k) if n > k else np.nan

        out.append({
            "Subset": subset_name,
            "Variable": var,
            "Groups": k,
            "Statistic (H)": round(H,3),
            "p-value": round(p,6),
            "η²": round(eta2,4),
            "Significant (p<0.05)": p<0.05
        })
    return pd.DataFrame(out)

# ---- run for all subsets ----
res = []
for name, idx in subsets.items():
    df_sub = df.loc[idx]
    if not df_sub.empty:
        res.append(run_kw(df_sub, name))

results_df = pd.concat(res, ignore_index=True)
results_df.sort_values(["Subset","p-value"], inplace=True)

# save or view
results_df


,Subset,Variable,Groups,Statistic (H),p-value,η²,Significant (p<0.05)
1,ALL,cdv_bin,3,61.586,0.000000,0.1118,True
2,ALL,size,2,27.551,0.000000,0.0497,True
3,ALL,fire_region,2,11.111,0.000858,0.0189,True
5,ALL,desert_region,2,7.643,0.005699,0.0124,True
4,ALL,mountain_region,2,1.373,0.241320,0.0007,False
0,ALL,monitor_bin,2,1.338,0.247422,0.0006,False
6,ALL,urban_category,2,0.012,0.913380,-0.0019,False
43,FN,cdv_bin,2,21.411,0.000004,0.4082,True
44,FN,size,2,14.566,0.000135,0.2713,True
45,FN,fire_region,2,10.224,0.001386,0.1845,True


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal

# ---- subset definitions ----
subsets = {
    "ALL": df.index,
    "TP_TN": df.index[df["classification"].isin(["TP","TN"])],
    "FP_FN": df.index[df["classification"].isin(["FP","FN"])],
    "TP": df.index[df["classification"] == "TP"],
    "TN": df.index[df["classification"] == "TN"],
    "FP": df.index[df["classification"] == "FP"],
    "FN": df.index[df["classification"] == "FN"],
}

group_vars = [
    "monitor_bin",
    "cdv_bin",
    "size",
    "fire_region",
    "mountain_region",
    "desert_region",
    "urban_category",
    "monitor_coverage_category"
]

def run_kw(df_sub, subset_name, value_col):
    out = []
    for var in group_vars:
        temp = df_sub[[var, value_col]].dropna()
        groups = temp[var].unique()
        if len(groups) < 2:
            continue

        # Kruskal–Wallis test
        data_groups = [temp.loc[temp[var] == g, value_col] for g in groups]
        H, p = kruskal(*data_groups)

        # η² effect size
        n = len(temp)
        k = len(groups)
        eta2 = (H - k + 1) / (n - k) if n > k else np.nan

        out.append({
            "Subset": subset_name,
            "Variable": var,
            "Groups": int(k),
            "Statistic (H)": round(float(H), 3),
            "p-value": round(float(p), 6),
            "η²": round(float(eta2), 4) if not np.isnan(eta2) else np.nan,
            "Significant (p<0.05)": bool(p < 0.05)
        })
    return pd.DataFrame(out)

# ---- run for all subsets ----
res = []
for name, idx in subsets.items():
    df_sub = df.loc[idx]
    if df_sub.empty:
        continue

    # Use signed 'diff' for single classes (TP/TN/FP/FN), else 'abs_diff'
    value_col = "diff" if name in {"TP", "TN", "FP", "FN"} else "abs_diff"
    res.append(run_kw(df_sub, name, value_col))

results_df = pd.concat(res, ignore_index=True)
results_df.sort_values(["Subset", "p-value"], inplace=True)

# save or view
# results_df.to_csv("kruskal_eta2_by_subset_signed_or_abs.csv", index=False)
results_df


,Subset,Variable,Groups,Statistic (H),p-value,η²,Significant (p<0.05)
1,ALL,cdv_bin,3,61.586,0.000000,0.1118,True
2,ALL,size,2,27.551,0.000000,0.0497,True
3,ALL,fire_region,2,11.111,0.000858,0.0189,True
5,ALL,desert_region,2,7.643,0.005699,0.0124,True
4,ALL,mountain_region,2,1.373,0.241320,0.0007,False
0,ALL,monitor_bin,2,1.338,0.247422,0.0006,False
6,ALL,urban_category,2,0.012,0.913380,-0.0019,False
43,FN,cdv_bin,2,21.411,0.000004,0.4082,True
44,FN,size,2,14.566,0.000135,0.2713,True
45,FN,fire_region,2,10.224,0.001386,0.1845,True
